## 列車運行情報を自分のLINEアカウントに送信する（一回実行用）

In [1]:
# japanize_matplotlibが未インストール状態の場合に、コメント解除して本セルを単独で実行する（先頭に!が必要）
#!pip install japanize-matplotlib
#!pip install schedule

In [2]:
# Googleドライブがマウントされていない場合に、2行のコメント解除して本セルを単独で実行する
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
# # matplotlibで日本語を使えるようにする
# import japanize_matplotlib

In [4]:
import urllib.request as req
from bs4 import BeautifulSoup
import pandas as pd
import requests
#import schedule
from time import sleep
from datetime import datetime, date

## 初期設定

In [5]:
y_train_1 = ['京浜東北根岸線','湘南新宿ライン','東海道本線','横須賀線']
y_train_2 = ['横浜線','東急東横線','京急本線','ブルーライン']
y_train_3 = ['相鉄線','みなとみらい線']
y_train = y_train_1 + y_train_2 + y_train_3
url = "https://transit.yahoo.co.jp/traininfo/area/4/"

# 定期実行する関数を準備

In [6]:
def task():
    global url
    res = req.urlopen(url)
    soup = BeautifulSoup(res, "lxml")
    result = []
    table = soup.select("table")
    for table_list in table:
        tr_list = table_list.find_all("tr")
        for tr in tr_list:
            result_row = []
            td_list = tr.find_all(["td", "th"])
            for td in td_list:
                cell = td.get_text()
                result_row.append(cell)
            result.append(result_row)
    # スクレイピングの結果のリストをデータフレームに変換する
    df1 = pd.DataFrame(result[1:][:])
    df1.columns = result[0]
    # 途中にある見出し行を取り除く
    df2 = df1[df1['路線'] != '路線']
    # 横浜駅に乗り入れている路線に限定する
    df3 = df2[df2['路線'].isin(y_train)]
    # テキストメッセージの作成
    ch_msg_1 = '【横浜駅乗り入れ鉄道路線運行状況】'
    ch_msg_2 = '以下記述がなければ全て平常運転です。'
    ch_msg = ch_msg_1 + ch_msg_2
    for irow in range(len(df3)):
      if df3.iloc[irow,1] != '平常運転':
        msg_append = '〔路線〕'+df3.iloc[irow,0]+'　〔状況〕'+df3.iloc[irow,1]+'　〔詳細〕'+df3.iloc[irow,2]
        ch_msg += msg_append
    # LINE Notifyを用いて自分のアカウントに送信
    current_hour = datetime.now().hour
    url = "https://notify-api.line.me/api/notify"
    access_token = 'zVCF9ov2dpki4b7G3vodGTE6XyAX5n2NJ8a1OTKWunp'
    headers = {'Authorization': 'Bearer ' + access_token}
    payload = {'message': ch_msg}
    r = requests.post(url, headers=headers, params=payload,)

# 関数の実行

In [7]:
task()

# 以下はスケジュール実行する場合のみ

In [8]:
# # スケジュール登録
# schedule.every(1).minutes.do(task)
# # 何時から実行するか定義
# year = date.today().year
# month = date.today().month
# hour = 6
# minute = 0
# second = 0
# set_from_time = datetime(year,month,date,today().day,hour,minute,second)
# # イベント実行（特定の時間が来てから定期実行
# while datetime.now() >= set_from_time:
#   schedule.run_pending()
#   sleep(1)